# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [114]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.844216695135329, 4.716258277002348, 6.025366749890605, 4.755038908779573, 4.911697602463441, 3.8478310497651957, 3.7595943235876996, 6.074171098785609, 4.06054717142707, 4.490001195719794]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [115]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.768375888354907, 7.350391696456876, 8.539638909053721, 7.040251388248902, 7.403072003308161, 7.476647540564181, 9.17405057025647, 6.059744343083684, 8.8891590816516, 8.88409310831863]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [116]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1[-1]
        s1.pop()
        data.append(p1)
    else:
        p2 = s2[-1]
        s2.pop()
        data.append(p2)
print(data)

[4.490001195719794, 4.06054717142707, 8.88409310831863, 8.8891590816516, 6.074171098785609, 3.7595943235876996, 6.059744343083684, 9.17405057025647, 7.476647540564181, 3.8478310497651957]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1. No of components = 2.
2. Means of components = [5, 8]
3. Standard deviations of components = [1, 1]
4. Weight of the components = [1/2, 1/2] (Uniform Distribution)



d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [117]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))
print(data)
s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s1 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.490001195719794, 4.06054717142707, 8.88409310831863, 8.8891590816516, 6.074171098785609, 3.7595943235876996, 6.059744343083684, 9.17405057025647, 7.476647540564181, 3.8478310497651957]
[8.88409310831863, 8.8891590816516, 9.17405057025647, 7.476647540564181]
[8.768375888354907, 7.350391696456876, 8.539638909053721, 7.040251388248902, 7.403072003308161]
P(S_1) = 0.5454545454545454,  P(S_2) = 0.45454545454545453
mean_1 = 4.715314863728175,  mean_2 = 7.820345977084514
var_1 = 0.9665662984535758,  var_2 = 0.4839330394492318


c:\Users\91960\anaconda3\envs\tf\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


- The mean of the distributions clustered are close to the true values.
- The variances are almost close to the true values.
- The weights of the distributions are close to the true values.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [118]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], np.sqrt(var[j])))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        #prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x )
    prob_s1_x.append( prob_s[1] * pdf_i[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.490001195719794
probability of observing that point if it came from cluster 0 =  0.395265743710524
probability of observing that point if it came from cluster 1 =  6.049910721613549e-06
point =  4.06054717142707
probability of observing that point if it came from cluster 0 =  0.3250712242937205
probability of observing that point if it came from cluster 1 =  2.602949244402122e-07
point =  8.88409310831863
probability of observing that point if it came from cluster 0 =  5.058485933309215e-05
probability of observing that point if it came from cluster 1 =  0.17814441299400555
point =  8.8891590816516
probability of observing that point if it came from cluster 0 =  4.949093919820523e-05
probability of observing that point if it came from cluster 1 =  0.17616699008159412
point =  6.074171098785609
probability of observing that point if it came from cluster 0 =  0.1561222088201446
probability of observing that point if it came from cluster 1 =  0.02456594124454513
point =  3.7595

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [119]:
prob_s = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum( [x[0] * (x[1]-mean[0])**2 for x in zip(prob_s0_x,data)])/sum(prob_s0_x)  , sum( [x[0] * (x[1]-mean[1])**2 for x in zip(prob_s1_x,data)])/sum(prob_s1_x)  ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5794464784780239,  P(S_2) = 0.42055352152197634
mean_1 = 4.6722354656486536,  mean_2 = 8.475196089731227
var_1 = 0.954840003593836,  var_2 = 0.7391232584163143


- The means of the distributions are close to the K means values.
- The variances are close to the K means values.
- The weights of the distributions are close to the K means values.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [120]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], np.sqrt(var[j])))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        #prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x )
    prob_s1_x.append( prob_s[1] * pdf_i[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.490001195719794
probability of observing that point if it came from cluster 0 =  0.40122906275722303
probability of observing that point if it came from cluster 1 =  1.0014730557627847e-05
point =  4.06054717142707
probability of observing that point if it came from cluster 0 =  0.33562451880113636
probability of observing that point if it came from cluster 1 =  8.726413852912789e-07
point =  8.88409310831863
probability of observing that point if it came from cluster 0 =  3.77240176276184e-05
probability of observing that point if it came from cluster 1 =  0.4144105874993585
point =  8.8891590816516
probability of observing that point if it came from cluster 0 =  3.688987808829346e-05
probability of observing that point if it came from cluster 1 =  0.41324361496951023
point =  6.074171098785609
probability of observing that point if it came from cluster 0 =  0.14587243610531103
probability of observing that point if it came from cluster 1 =  0.009394502400731546
point =  3.

There are very slight differences between the values of the probabilities, but the hard assignment will be the same in this case.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [121]:
hard_assignment = np.round(prob_s1_x)
for x in zip(hard_assignment,data):
    print(x[1]," belongs to ",x[0])

4.490001195719794  belongs to  0.0
4.06054717142707  belongs to  0.0
8.88409310831863  belongs to  1.0
8.8891590816516  belongs to  1.0
6.074171098785609  belongs to  0.0
3.7595943235876996  belongs to  0.0
6.059744343083684  belongs to  0.0
9.17405057025647  belongs to  1.0
7.476647540564181  belongs to  1.0
3.8478310497651957  belongs to  0.0


Even upon visual inspection, we could clearly say that the assignments to the distributions are correct.